# RNN in tensorflow

In [1]:
import tensorflow as tf
import numpy as np
import random

In [2]:
# import text and extract chars
text = open('fdp.txt', 'r').read()
unique_chars = list(set(text))
text_size, vocab_size = len(text), len(unique_chars)
print('data has %d characters, %d unique.' % (text_size, vocab_size))
char_to_ix = {ch:i for i,ch in enumerate(unique_chars)}
ix_to_char = {i:ch for i,ch in enumerate(unique_chars)}

data has 12175 characters, 72 unique.


In [3]:
# Hyperparameters of RNN
hidden_size = 100
seq_length = 25

In [4]:
def one_hot(v):
    '''
    Convert character labels to one-hot variants
    :param v - index in char_to_index of character to convert
    :returns - 0-Vector with 1 at index v
    '''
    return np.eye(vocab_size)[v]

In [5]:
# defining the network
# start with parameters
x = tf.placeholder(tf.float32, [None, vocab_size]) # 25x76 = seq_length x vocab_size
y_in = tf.placeholder(tf.float32, [None, vocab_size]) # 25x76 = seq_length x vocab_size
h_start = tf.placeholder(tf.float32, [1, hidden_size]) # 1x100 

In [6]:
# define architecture
initializer = tf.random_normal_initializer(stddev=0.1)
with tf.variable_scope("RNN") as scope:
    h_state = h_start # initialize hidden state
    y_out_all = [] # collect each of y values choosen by network
    # split input of seq_length into single inputs x_in with t as timestep
    for t, x_in in enumerate(tf.split(x, seq_length,0)):
        if t > 0: 
            # reuse same variables within one iteration of training (one seq_length)
            scope.reuse_variables()
        Wxh = tf.get_variable('Wxh', [vocab_size, hidden_size], initializer=initializer)
        Whh = tf.get_variable('Whh', [hidden_size, hidden_size], initializer=initializer)
        Why = tf.get_variable('Why', [hidden_size, vocab_size], initializer=initializer)
        bh = tf.get_variable('bh', [hidden_size], initializer=initializer)
        by = tf.get_variable('by', [vocab_size], initializer=initializer)
        
        # forward-pass through first hidden layer
        h_state = tf.tanh(tf.matmul(x_in, Wxh) + tf.matmul(h_state, Whh) + bh)
        # forward-pass through output layer
        y_out = tf.matmul(h_state, Why) + by
        y_out_all.append(y_out)

In [9]:
# store last hidden state and calculate loss of all predicted y_out_all
h_last = h_state
output_softmax = tf.nn.softmax(y_out_all[-1])  # only for sampling
outputs = tf.concat(y_out_all, 0)
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=outputs, labels=y_in))

In [11]:
# minimize loss by using backprop and sgd through adam optimizer
minimizer = tf.train.AdamOptimizer()
grads_and_vars = minimizer.compute_gradients(loss)

# prevent gradient vanishing and explosion
grad_clipping = tf.constant(5.0, name='grad_clipping')
clipped_grads_and_vars = []
for grad, var in grads_and_vars:
    clipped_grad = tf.clip_by_value(grad, -grad_clipping, grad_clipping)
    clipped_grads_and_vars.append((clipped_grad, var))
    
updates = minimizer.apply_gradients(clipped_grads_and_vars)

In [13]:
# sampling from the network to see progress (in textual form)
def sample_network():
    sample_length = 200
    start_ix = random.randint(0, len(text) - seq_length)
    sample_seq_ix = [char_to_ix[ch] for ch in text[start_ix:start_ix + seq_length]]
    ixes = []
    sample_prev_state_val = np.copy(h_start_val)
    
    for t in range(sample_length):
        sample_input_vals = one_hot(sample_seq_ix)
        sample_output_softmax_val, sample_prev_state_val = sess.run([output_softmax, h_last], feed_dict={x: sample_input_vals, h_start: sample_prev_state_val})
        ix = np.random.choice(range(vocab_size), p=sample_output_softmax_val.ravel())
        ixes.append(ix)
        sample_seq_ix = sample_seq_ix[1:] + [ix]
    
    txt = ''.join(ix_to_char[ix] for ix in ixes)
    print('-----\n %s \n ----\n' % txt)

In [14]:
# run tensorflow session
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [17]:
# set starting position in text and number of iterations
position_in_text = 0
number_of_iterations = 0
total_iterations = 10000
h_start_val = np.zeros([1,hidden_size])

In [18]:
# loop for going through document and see if end was reached if not train again
while number_of_iterations < total_iterations:
    if position_in_text + seq_length + 1 >= len(
            text) or number_of_iterations == 0:
        # reset value of hidden state
        h_start_val = np.zeros([1, hidden_size])
        position_in_text = 0

    inputs = one_hot([
        char_to_ix[ch]
        for ch in text[position_in_text:position_in_text + seq_length]
    ])
    targets = one_hot([
        char_to_ix[ch]
        for ch in text[position_in_text + 1:position_in_text + seq_length + 1]
    ])
    h_start_val, loss_val, _ = sess.run([h_last, loss, updates], feed_dict={x:inputs, y_in:targets, h_start:h_start_val})
    
    # sample every 500 iterations
    if number_of_iterations % 500 == 0:
        print('iter: %d, p: %d, loss: %f' %(number_of_iterations, position_in_text, loss_val))
        sample_network()
        
    position_in_text += seq_length
    number_of_iterations += 1

iter: 0, p: 0, loss: 3.195930
-----
 htdn  oinen Uin äescheen.
dun  Eübendr.
Dmlegesa dsn sinn schlde 1ichiendn kZschldgen siit Sn.
Znd ch-npil ian ernlgengen uans Denden venseie benden.
Suwichtchuus en:tNmet EiölEnzBhndsnes daentenHwhnu 
 ----

iter: 500, p: 350, loss: 2.026916
-----
 t men LaGüe Zaso hin nit dir “ugüfte-enft.
Mir ge dit Gengen tibeiln fer gze  füe gen un ku sin Vorlen mnde Erropl sallen Mabeao san an kiulen Alfe foolenn In eln dan di.
Dasoen sobwie Heut, ie  eund  
 ----

iter: 1000, p: 700, loss: 1.772866
-----
 lbden.
Zndern
olineln tändegein: Unierserfen.ichen ehSch ben ich mid eunderefennecht schumeder er:-Fue .unsenznbehten Alder soaden hatchenichtelng den sbeldermelann senebtn Zinden.
Uäder undernelt aäw 
 ----

iter: 1500, p: 1050, loss: 1.934769
-----
 u1ndern.
Lis kapn.
EUNO9NIS5HzO Firlgen Lah eltebthen schen Soe Lneserenn deremzhlren solaus worten Sichändase nerNes ein Shumgt AU vom Foren.
0Men BlTsch fen.
Int dnischenPm?er komnen
Westger edar  
 ----

i